In [14]:
import geopandas as gpd

# Shapefile path
shapefile_path = '../src/models/area_optimization/data/raw/shp/MA0001_2020_2020_08/rcom.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Convert to GeoJSON
geojson = gdf.to_json()

# Save GeoJSON to file
geojson_path = '../src/models/area_optimization/data/processed/geojson/rcom.geojson'
with open(geojson_path, 'w') as f:
    f.write(geojson)

In [ ]:
!pip install chardet

In [ ]:
import json
import chardet

def decode_sjis_text(text):
    """Shift-JISでエンコードされた文字列をデコードする"""
    try:
        return bytes(text, 'latin1').decode('shift-jis')
    except UnicodeDecodeError:
        return text

def decode_geojson_file(input_path: str, output_path: str = None) -> dict:
    """
    GeoJSONファイルを読み込んでデコードする関数
    
    Args:
        input_path (str): 入力GeoJSONファイルのパス
        output_path (str, optional): 出力ファイルのパス。指定しない場合は保存しない
    
    Returns:
        dict: デコードされたGeoJSONデータ
    """
    # エンコーディング候補
    encodings = [
        'shift-jis',      # Windows日本語
        'utf-8'           # UTF-8
    ]
    
    # ファイルの文字コードを自動検出
    with open(input_path, 'rb') as f:
        raw_data = f.read()
        detected = chardet.detect(raw_data)
        encodings.insert(0, detected['encoding'])
    
    for encoding in encodings:
        try:
            with open(input_path, 'r', encoding=encoding) as f:
                content = f.read()
                data = json.loads(content)
                
                # GeoJSON形式の検証
                if not isinstance(data, dict) or data.get('type') != 'FeatureCollection':
                    continue
                
                # 日本語文字列のデコード処理
                for feature in data['features']:
                    props = feature.get('properties', {})
                    # 特定のキーの文字列をShift-JISでデコード
                    for key in ['PREF_NAME', 'CITY_NAME', 'KCITY_NAME', 'RCOM_NAME', 'RCOM_KANA']:
                        if key in props and isinstance(props[key], str):
                            props[key] = decode_sjis_text(props[key])
                
                # 文字化けチェック
                has_encoding_error = False
                for feature in data['features']:
                    props = feature.get('properties', {})
                    for value in props.values():
                        if isinstance(value, str):
                            if any(c in value for c in ['�', '?', '\ufffd']):
                                has_encoding_error = True
                                break
                    if has_encoding_error:
                        break
                
                if not has_encoding_error:
                    print(f"検出された文字コード: {encoding}")
                    
                    # 出力パスが指定されている場合は保存
                    if output_path:
                        with open(output_path, 'w', encoding='utf-8') as f:
                            json.dump(data, f, ensure_ascii=False, indent=2)
                        print(f"ファイルを保存しました: {output_path}")
                    
                    return data
                
        except (UnicodeDecodeError, json.JSONDecodeError):
            continue
    
    raise ValueError("適切な文字コードでデコードできませんでした。")

# 使用例
input_path = '../src/models/area_optimization/data/processed/geojson/rcom.geojson'
output_path = '../src/models/area_optimization/data/processed/geojson/rcom_decoded.geojson'

try:
    decoded_data = decode_geojson_file(input_path, output_path)
    print("データのデコードに成功しました")
except Exception as e:
    print(f"エラーが発生しました: {str(e)}")

検出された文字コード: ascii
ファイルを保存しました: ../src/models/area_optimization/data/processed/geojson/rcom_decoded.geojson
データのデコードに成功しました
